In [7]:
H = 4.
c = 10.e3
phi = 30.
alpha = 10.
beta = 10.
delta = 5
gamma = 16.e3
import numpy as np

In [8]:
def slip_length(theta):
    L = H * (1 + np.tan(alpha) * np.tan(beta))/(np.sin(theta) - np.cos(theta)*np.tan(beta))
    return L

In [9]:
def area(theta):
    L = slip_length(theta)
    A = np.array([0,0])
    B = np.array([- H * np.tan(alpha), H])
    C = np.array([L * np.cos(theta), L*np.sin(theta)])
    term1 = (B[0]*C[1] - C[0] * B[1])
    term2 = (A[0]*C[1] - C[0] * A[1])
    term3 = (A[0]*B[1] - B[0] * A[1])
    return np.abs((term1 - term2 + term3))

In [10]:
def cohesive_force(theta):
    return c * slip_length(theta)

In [11]:
def weight(theta):
    return gamma*area(theta)

In [12]:
def Eact(theta):
    term1 = weight(theta) * np.sin(theta-phi) - cohesive_force(theta) * np.cos(phi)
    term2 = np.cos(alpha+delta+phi-theta)
    return term1/term2

In [13]:
def dEa_dtheta(theta,eps=1.e-8):
    return (Eact(theta+eps) - Eact(theta-eps))/(2.*eps)

In [14]:
def residual(theta):
    return dEa_dtheta(theta)

In [15]:
def dr_dtheta(theta,eps=1.e-8):
    return (residual(theta+eps) - residual(theta-eps))/(2.*eps)

In [19]:
def find_theta(theta_0=np.pi/6, tol=1.e-5):
    theta = theta_0
    iteration_counter = 0
    while (np.abs(residual(theta)) > tol and iteration_counter < 100):
        theta -= residual(theta) / dr_dtheta(theta)
        iteration_counter += 1
    return theta, iteration_counter

NameError: name 'np' is not defined

In [17]:
def theta_ag_norm():
    t1 = np.cos(phi - alpha)
    t2 = np.sin(phi - alpha) + np.sqrt((np.sin(phi + delta)*np.cos(alpha-beta))/(np.sin(phi-beta)*np.cos(alpha+delta)))
    print("For the non-cohesive case the norm gives us %.2f °." %((phi + np.arctan(t1/t2))*180/np.pi))

In [18]:
from ipywidgets import widgets
from ipywidgets import interact

def plot_interactive_earth_pressure():
    @interact(ic=widgets.IntSlider(min=0, max=50, value=10, step=5, description='$c$ / kN m$^{-2}$:'),
              iphi = widgets.IntSlider(min=20, max=45, value=30, step=5, description='$\\varphi$ / °:'),
              igamma=widgets.IntSlider(min=14, max=25, value=18, step=1, description='$\\gamma$ / kN m$^{-3}$:'),
              iH=widgets.IntSlider(min=2, max=10, value=3, step=1, description='$H$ / m:'),
              ialpha = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\alpha$ / °:'),
              ibeta = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\beta$ / °:'),
              idelta = widgets.IntSlider(min=0, max=20, value=10, step=2, description='$\\delta_\\mathrm{a}$ / °:'),
             )
    def plot_earth_pressure(ic,iphi,igamma,iH,ialpha,ibeta,idelta):
        #unit conversion
        global c
        global gamma
        global phi
        global alpha
        global beta
        global delta
        global H

        H = iH
        c = ic * 1000.
        gamma = igamma * 1000.
        phi = iphi * np.pi/180.
        alpha = ialpha * np.pi/180.
        beta = ibeta * np.pi/180.
        delta = idelta * np.pi/180.
        #Calculation
        theta_a, it = find_theta(np.pi/4 + phi/2)

        thetas = np.linspace(np.pi/6,np.pi/2*0.9,100)
        plt.figure(figsize=(14,8))
        plt.plot(thetas*180/np.pi,Eact(thetas)/1e6)
        plt.plot(theta_a*180/np.pi,Eact(theta_a)/1e6,marker='d',color='red')
        plt.xlabel('$\\vartheta$ / °')
        plt.ylabel('$E_\\mathrm{a}$ / MN m$^{-1}$')
        plt.text(theta_a*180/np.pi,Eact(theta_a)/1e6,'$E_\\mathrm{a} = %.2f$ MN m$^{-1}$, $\\vartheta$ = %.1f °' %(Eact(theta_a)/1e6,theta_a*180/np.pi))
        
        plt.show();